# Capstone Assignment Notebook


## Introduction

This is a notebook for the assigments of **Appied Data Science Capstone** [*IBM Data Science course 9*](https://www.coursera.org/learn/applied-data-science-capstone)


## Greeting to Capstone Project Course

Import primary modules: **pandas** and **numpy**

In [2]:
import pandas as pd # primary data structure library
import numpy as np  # useful for many scientific computing in Python

Print the following the statement: *Hello Capstone Project Course!*

In [3]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


## Segmenting and Clustering Neighborhoods in Toronto

### Scraping data table and transformation

In [4]:
# We will use pandas to import table from the provided wiki url
wiki = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

In [48]:
# There are 3 items in this list
len(wiki)

3

In [5]:
# The table we need is the first item in the list
PBN_table = wiki[0]

In [6]:
# Have a look of the dataframe and we need to do some cleanups
PBN_table.head()

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [7]:
# First, we assign the first row as column name
header = PBN_table.iloc[0] # store header into a new variable
PBN_table = PBN_table[1:] # remove the first row
PBN_table.columns = header # assign header
PBN_table.head()

,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


In [22]:
# Remove cells with a borough that is not assigned.
PBN_table_temp=PBN_table.drop(PBN_table[PBN_table["Borough"] == "Not assigned"].index)
PBN_table_temp.head()

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor


In [82]:
# Aggreagte rows with the same postcode, the neighbourhoods should be separated by comma.
aggregate_function = lambda a: ", ".join(a) # create a function to join strings

# The aggregate() function on a dictionary, where the 1st item is column name and the 2nd item is the function to apply
# Since groupby automatically set the called column as index, we use reset_index and drop = True to set the index back
# to integer
PBN_table_temp1 = PBN_table_temp.groupby(by='Postcode').aggregate({"Postcode" : "first",
                                                                   "Borough" : "first",
                                                                   "Neighbourhood": aggregate_function}).reset_index(drop=True)
PBN_table_temp1.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [83]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
# First check the row with column of "Neighbourhood" that has value "Not assigned"
print(PBN_table_temp1.loc[PBN_table_temp1['Neighbourhood'] == 'Not assigned'])

   Postcode       Borough Neighbourhood
85      M7A  Queen's Park  Not assigned


In [84]:
# Change its value with its borough value
PBN_table_temp1.loc[PBN_table_temp1['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = PBN_table_temp1.loc[PBN_table_temp1['Neighbourhood'] == 'Not assigned', 'Borough']
# And check again to see if any column of "Neighbourhood" that has value "Not assigned"
print(PBN_table_temp1.loc[PBN_table_temp1['Neighbourhood'] == 'Not assigned'])

Empty DataFrame
Columns: [Postcode, Borough, Neighbourhood]
Index: []


In [85]:
# Assign the last temp df to df and show its dimention
df = PBN_table_temp1
df.shape

(103, 3)

### Adding geographical coordinates

In [86]:
# install required package
!pip install geocoder

     |████████████████████████████████| 102kB 427kB/s ta 0:00:011
     |████████████████████████████████| 829kB 3.7MB/s eta 0:00:01
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=493313 sha256=e1b4d62260c21dd54c33a37c71ffb3801a8969055d5149db24205f5e096aaaf4
  Stored in directory: /Users/jun/Library/Caches/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future


In [1]:
import geocoder # import geocoder

In [ ]:
# Steup two empty lists to store latitude and longitude data
Latitude = []
Longitude = []

# Using a for loop to fill up the lists
for postal_code in df["Postcode"]:
    
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng

    Latitude.append(lat_lng_coords[0])
    Longitude.append(lat_lng_coords[1])

# Assigning those two lists to the df
df["Latitude"] = Latitude
df["Longitude"] = Longitude

df.head()

In [101]:
# Since I failed to get coordinates from google, I used the provided data instead
!wget -O Canada_Geospatial_data https://cocl.us/Geospatial_data

--2019-11-22 20:55:21--  https://cocl.us/Geospatial_data
Resolving cocl.us... 169.48.113.194
Connecting to cocl.us|169.48.113.194|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-11-22 20:55:23--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com... 103.116.4.197
Connecting to ibm.box.com|103.116.4.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-11-22 20:55:24--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-11-22 20:55:25--  https://ibm.ent.box.com/public/static/

In [102]:
# Store the geospatial_data in a dataframe
Geospatial_data = pd.read_csv("Canada_Geospatial_data", delimiter=",")
Geospatial_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [106]:
Geospatial_data.shape

(103, 3)

In [108]:
# Merge two dataframes based on the key column
df_with_cord = pd.merge(left=df, right=Geospatial_data, left_on="Postcode", right_on="Postal Code")
df_with_cord.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [109]:
# Remove the redundant column
df_with_cord.drop(columns="Postal Code", inplace=True)
df_with_cord.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [116]:
# Change column name as the same as shown in the course page
df_with_cord.rename(columns={"Postcode":"PostalCode"}, inplace=True)
df_with_cord.head(n=12)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
